In [ ]:
import numpy as np
import pandas as pd
import re
import datetime
from datetime import date
import matplotlib.mlab as mlab
import matplotlib.pyplot as pyplot
import math

## Load log data and tag data

In [ ]:
logDF = pd.read_csv('./dataframes/historicLogData.csv')
tagDF = pd.read_csv('./dataframes/tagData.csv')

In [ ]:
logDF=logDF.dropna()

In [ ]:
logDF.head()

In [ ]:
tagDF.head()

## Prepare data for the aggregation
#### (delete -1s)

In [ ]:
badData = logDF[logDF.usage_cpu==-1]

In [ ]:
# lets see the distribution
b=badData.groupby('instance', as_index=False).agg('count')
b

In [ ]:
logDF=logDF[logDF.usage_cpu!=-1]

## Max and count times > x % throughout whole time period

In [ ]:
aggregations = {
    'usage_cpu' : {
        'overall_avg':'mean',
        '95%p': lambda x: np.percentile(x, 95),
        'median': 'median',
        'overall_std':'std',
        'max_cpu': 'max',
        'p_over40': lambda x: (sum(x>40)/len(x))*100       # percentage of times that usage is over 40%
    }  
}

In [ ]:
cpu_stats=logDF.groupby('instance', as_index=False).agg(aggregations)   # as_index=F to drop extra row indexes
cpu_stats.columns = cpu_stats.columns.droplevel(level=0)                  # drop extra row indexes

cpu_stats.rename(columns={'': 'instance'}, inplace=True)
cpu_stats

In [ ]:
cpu_stats.describe()

In [ ]:
(cpu_stats[cpu_stats['95%p']>50])    # the ones with 95th percentile over 50%

## Binning every quarter

In [ ]:
x = cpu_stats['95%p']
num_bins = 4                          
(n,bins,patches)=pyplot.hist(x, num_bins, facecolor='blue', alpha=0.5)
pyplot.show()

In [ ]:
n

## Binning every 5% usage

In [ ]:
num_bins = 20
(n,bins,patches)=pyplot.hist(x, num_bins, facecolor='blue')
pyplot.xlabel('95th percentile of cpu usage (%)')
pyplot.ylabel('number of instances')
pyplot.grid(True)
pyplot.title('Histogram of 95th percentile')
pyplot.savefig('./dataframes/95p')
pyplot.show()

In [ ]:
n

## Merge log statistics with tag data

In [ ]:
cpu_data = pd.merge(tagDF, cpu_stats, how='inner', left_on='Host', right_on='instance')
cpu_data.head()

In [ ]:
# Remove instance column
cpu_data.drop('instance', axis=1, inplace=True)

In [ ]:
# Add admin or no admin column (factor variable)
# cpu_data['adm']=cpu_data.Host.apply(lambda x: 1 if 'adm' in x else 0)

## Add instance type

In [ ]:
typeDF = pd.read_csv('./dataframes/instanceData.csv')
typeDF = typeDF[['Host','type']]
cpu_data = pd.merge(cpu_data, typeDF, how='inner', on='Host')      

In [ ]:
cpu_data.head()

In [ ]:
cpu_data.describe()

In [ ]:
# cpu_data.to_csv('./dataframes/allData.csv',sep=',',index=False, encoding="utf-8")

## Downgrade decision rule
#####      0-25%    -> 1/4 cpus
#####      25-50%  -> 1/2
#####      50-75%  -> 3/4
#####      75-100% -> consider upgrade

In [ ]:
cpu_data['downgrade']=cpu_data['95%p'].apply(lambda x: 1/4 if x<25 else 1/2 if x<50 else 3/4 if x<75 else 1)

In [ ]:
cpu_data

## Recommend new type

In [ ]:
aws = pd.read_excel('./AWS instances.xlsx')

In [ ]:
aws=aws[['Instance Type','vCPU*','Memory (GiB)']]
aws.columns=['type','cpus','memory']

In [ ]:
aws=aws.dropna()

In [ ]:
rec = pd.merge(cpu_data, aws, how='inner',on='type')
rec

In [ ]:
rec['rec_cpus']=(rec.downgrade*rec.cpus)
rec.rec_cpus=rec.rec_cpus.apply(lambda x: math.ceil(x))
rec

In [ ]:
rec.rec_cpus.unique()

In [ ]:
aws['family']=aws.type.apply(lambda x: x.split('.')[0])

## Recommend Type function 
##### (inside same family, if >1 option with = cpu then choose one with most memory)

In [ ]:
def recommendType(t, cpus):
    family = t.split('.')[0]
    test=aws[aws['family']==family]
    if cpus==1:
        cpus=min(test.cpus)
    test=test[test['cpus']==cpus]
    if len(test)>1:
        test=test[test.memory==max(test.memory)]
    if len(test)==0:
        return(t)
    test.reset_index(inplace=True)   
    rec_type = test.type[0]    
    return(test.type)

In [ ]:
rec['recom_type']=rec.apply(axis=1, func=lambda x: recommendType(x.type,x.rec_cpus))

### Instances recommended to change their type

In [ ]:
rec[rec.type!=rec.recom_type]

In [ ]:
cpu_data.type.unique()

In [ ]:
rec.recom_type.unique()

In [ ]:
rec[rec.type=='r3.large']

In [ ]:
rec[rec.type=='r3.xlarge']

In [ ]:
rec.to_csv('./dataframes/recommendations.csv')

## Recommended to upgrade

In [ ]:
rec[rec.downgrade==1]

In [ ]:
aux=rec[rec.downgrade==1]
x = aux['95%p']
num_bins = 4
(n,bins,patches)=pyplot.hist(x, num_bins, facecolor='blue', alpha=0.5)
pyplot.show()

## Cost Savings

In [ ]:
awsPrice = pd.read_csv('./awsPrices.csv')
awsPrice

In [ ]:
awsPrice=awsPrice[['type','Price3']]

In [ ]:
downgraded = rec[rec.type!=rec.recom_type]

In [ ]:
len(downgraded)  # how many were really recommended to downgrade

In [ ]:
rec

In [ ]:
before = rec.groupby('type', as_index=False).agg('count')
before = before[['type','cpus']]
before.columns = ['type', 'count']
before

In [ ]:
beforeCost = pd.merge(before, awsPrice, how='inner', on='type')
beforeCost

In [ ]:
beforeCost['total_b']=beforeCost['count']*beforeCost['Price3']
beforeCost

In [ ]:
after = rec.groupby('recom_type', as_index=False).agg('count')
after = after[['recom_type','cpus']]
after.columns = ['type', 'count']
after

In [ ]:
afterCost = pd.merge(after, awsPrice, how='inner', on='type')
afterCost['total_b']=afterCost['count']*afterCost['Price3']
afterCost

In [ ]:
beforeCost

In [ ]:
total_b = sum(beforeCost.total_b)
total_a = sum(afterCost.total_b)
savings = total_b-total_a
p_savings = round(savings/total_b*100,2)
print('The total cost before was: ',total_b,'\nThe total cost after is: ',total_a,'\nSavings are: ', savings, ' (',p_savings,'%) ')